In [157]:
from plot import *
from fft import *
from maths import rmse, mae, mape


ImportError: cannot import name 'mape' from 'maths' (/Users/bhavyemathur/Desktop/Projects/Spherindrical Fourier Transform/MERRA-2/wind-prediction/maths.py)

In [158]:
def dft3_at_level(data: np.ndarray, level: int, quantile: float = 0.75):
    if level in DFT3_LEVEL_CACHE:
        fft = DFT3_LEVEL_CACHE[level]
    else:
        fft = np.fft.rfftn(data)
        DFT3_LEVEL_CACHE[level] = fft

    amplitudes = np.abs(fft)

    fft_real = []
    fft_imag = []
    fft_i_indices = []
    fft_j_indices = []
    fft_k_indices = []

    cutoff_amp = np.quantile(amplitudes, quantile)

    last_i = 0
    last_j = 0
    last_k = 0

    for k in range(289):
        for j in range(361):
            for i in range(365 * 8):
                if amplitudes[i, j, k] < cutoff_amp:
                    continue

                fft_real.append(fft[i, j, k].real)
                fft_imag.append(fft[i, j, k].imag)

                while (di := i - last_i) > 253:
                    fft_i_indices.append(254)
                    last_i += 253

                while (dj := j - last_j) > 253:
                    fft_j_indices.append(254)
                    last_j += 253

                while (dk := k - last_k) > 253:
                    fft_k_indices.append(254)
                    last_k += 253

                if di < 0:
                    fft_i_indices.append(255)
                    last_i = 0
                else:
                    fft_i_indices.append(di)
                    last_i += di

                if dj < 0:
                    fft_j_indices.append(255)
                    last_j = 0
                else:
                    fft_j_indices.append(dj)
                    last_j += dj

                if dk < 0:
                    fft_k_indices.append(255)
                    last_k = 0
                else:
                    fft_k_indices.append(dk)
                    last_k += dk

    fft_real = np.array(fft_real, dtype="float32") / 32768
    fft_imag = np.array(fft_imag, dtype="float32") / 32768

    fft_i_indices = np.array(fft_i_indices, dtype="uint8")
    fft_j_indices = np.array(fft_j_indices, dtype="uint8")
    fft_k_indices = np.array(fft_k_indices, dtype="uint8")

    return fft_real.astype("float16"), fft_imag.astype("float16"), \
        fft_i_indices, fft_j_indices, fft_k_indices


def idft3_at_time_and_level(fft_real, fft_imag, fft_i_indices, fft_j_indices, fft_k_indices):
    ifft = np.zeros((365 * 8, 361, 289), dtype="complex64")
    fft = fft_real.astype("complex64") * 32768 + fft_imag.astype("complex64") * 32768j

    i = 0
    j = 0
    k = 0

    i_index = 0
    j_index = 0
    k_index = 0

    for idx in range(len(fft)):
        while (di := fft_i_indices[i_index]) == 254:
            i += 253
            i_index += 1

        while (dj := fft_j_indices[j_index]) == 254:
            j += 253
            j_index += 1

        while (dk := fft_k_indices[k_index]) == 254:
            k += 253
            k_index += 1

        if di == 255:
            i = 0
        else:
            i += di

        if dj == 255:
            j = 0
        else:
            j += dj

        if dk == 255:
            k = 0
        else:
            k += dk

        ifft[i, j, k] = fft[idx]

        i_index += 1
        j_index += 1
        k_index += 1

    return np.fft.irfftn(ifft)

In [159]:
def fit_dft3_at_level(filename: str, variable: str, level: int, **kwargs):
    print("Loading data...")
    data = load_variable_at_level(filename, variable, level)

    print("Performing DFT...")
    fft = dft3_at_level(data, level, **kwargs)

    print("Performing IDFT...")
    prediction = idft3_at_time_and_level(*fft)

    print(f"Original Stdev:  {data.astype('float32').std()} m/s")
    print(f"Predicted RMSE: {rmse(data, prediction)} m/s")
    print(f"Predicted MAPE: {mape(data, prediction)}%")
    print(f"Predicted MAE:  {mae(data, prediction)} m/s")
    print(f"Frequencies: {len(fft[0])}")
    print(f"Size/level: {sum(el.nbytes for el in fft if hasattr(el, 'nbytes')) / (1024 ** 2)} mB")
    print(f"Size/year: {sum(el.nbytes for el in fft if hasattr(el, 'nbytes')) * 72 / (1024 ** 2)} mB")

In [164]:
fit_dft3_at_level("MERRA2_100.tavg3_3d_asm_Nv.1980{:0>2}{:0>2}.nc4", "U",
                  level=71, quantile=0.994)


Loading data...
Performing DFT...
Performing IDFT...
Original Stdev:  6.392603397369385 m/s
Predicted RMSE: 1.0082440250483846 m/s


/var/folders/r5/mzzh3rn14lgb2__wr5h7swqm0000gn/T/ipykernel_92607/2292165165.py:2: RuntimeWarning: divide by zero encountered in divide
  ((data - prediction) / data).mean()
/Users/bhavyemathur/Desktop/Projects/Spherindrical Fourier Transform/MERRA-2/venv/lib/python3.11/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


Predicted MAPE: None%
Predicted MAE:  0.7204414327704877 m/s
Frequencies: 1827845
Size/level: 12.342144012451172 mB
Size/year: 888.6343688964844 mB
